In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from src.MM_estimation import estimate_w_factor_loading, calc_variance_of_default_rate
from src.ML_estimation import calculate_my_likelihood_arr
from src.ML_estimation import ml_estimation_linear_with_w_and_g
from src.ML_estimation import parameter_estimation
from src.variable_change import a_calc_func, b_calc_func, w_calc_func, gamma_calc_func
from src.ML_estimation import calculate_variable_changed_likelihood_arr
from src.sucess_probability import p_g
from scipy.optimize import minimize

In [7]:
# Load the data
grades_data = pd.read_csv('data/grades_hist_pd.csv', sep=';', index_col=0)

for col_name in grades_data.columns:
    grades_data[col_name] = grades_data[col_name] / 100
    
grades_cols = grades_data.columns

n_g_assumption = 3000

for col_name in grades_cols:
    grades_data[str(col_name) + " defaults"] = (n_g_assumption * grades_data[col_name])
    
num_of_rows = grades_data.shape[0]
n_g_over_time = np.array([n_g_assumption] * grades_data.shape[0])

# Choose last four columns and calculate the sum of defaults
d_g = grades_data.iloc[:, -5:].sum(axis=0).tolist()
d_g = np.array([int(round(x)) for x in d_g])
n_g = np.array([n_g_assumption * grades_data.shape[0]] * len(d_g))

In [8]:
MM_params = {}
for grade in ["A", "BBB", "BB", "B", "CCC/C"]:
    MM_params[grade] = estimate_w_factor_loading(grades_data[grade], n_g_over_time, initial_guess=0.45)
    print(f"Estimated parameters with method of moments for grade {grade}:", MM_params[grade][0], norm.ppf(MM_params[grade][1]))

MM_gammas = [norm.ppf(MM_params[grade][1]) for grade in ["A", "BBB", "BB", "B", "CCC/C"]]
MM_ws = [MM_params[grade][0] for grade in ["A", "BBB", "BB", "B", "CCC/C"]]

Estimated parameters with method of moments for grade A: 0.4499803380652506 -3.274097580352716
Estimated parameters with method of moments for grade BBB: 0.3133851305078121 -2.8865426119490847
Estimated parameters with method of moments for grade BB: 0.34959246714062564 -2.3842278912651227
Estimated parameters with method of moments for grade B: 0.3288445137084037 -1.728911139866942
Estimated parameters with method of moments for grade CCC/C: 0.36077988602478867 -0.6770330266259245


In [14]:
n_g_input = [240000, 240000, 240000]
d_g_input = [662, 1421, 8284]

pd_list = [0.005, 0.0100, 0.0500]
gamma_list = norm.ppf(pd_list)

w_values = np.linspace(0.0, 0.5, 6)
params_dict = {}

for w in w_values:
    ML_linear_estimation = ml_estimation_linear_with_w_and_g(d_g_input, n_g_input, np.array([w]), gamma_list, fixed_w=True)
    params_dict[w] = (ML_linear_estimation[0], ML_linear_estimation[1], ML_linear_estimation[2].fun)
    print("Estimated parameters with maximum likelihood method:", ML_linear_estimation[0], ML_linear_estimation[1])
    print("Current function value:", ML_linear_estimation[2].fun)

C:\Users\kissp\PycharmProjects\VasicekPDModell\src\ML_estimation.py:271: RuntimeWarning: divide by zero encountered in log
  objective_function = lambda params: -np.log(calculate_variable_changed_likelihood_arr(


Estimated parameters with maximum likelihood method: [0.] [-2.77516819 -2.51686841 -1.81819665]
Current function value: 14.126046167198819
Estimated parameters with maximum likelihood method: [0.1] [-2.76128767 -2.50420115 -1.80907284]
Current function value: 17.32613222730679
Estimated parameters with maximum likelihood method: [0.2] [-2.71911474 -2.46595269 -1.78143632]
Current function value: 18.034038948510027
Estimated parameters with maximum likelihood method: [0.3] [-2.647421   -2.40093983 -1.73448903]
Current function value: 18.466132338039767


C:\Users\kissp\PycharmProjects\VasicekPDModell\src\ML_estimation.py:213: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  result, _ = quad(integrand, -3, 3, epsabs=1.49e-28)


Estimated parameters with maximum likelihood method: [0.4] [-2.54358118 -2.30677081 -1.66646408]
Current function value: 18.793795121156126
Estimated parameters with maximum likelihood method: [0.5] [-2.4034316  -2.17966591 -1.57463297]
Current function value: 19.07358420612712


In [15]:
-np.log(calculate_my_likelihood_arr(
            d_g_input, n_g_input, p_g, norm.pdf, np.array([0.45]), gamma_list))

51.32282976616954

In [17]:
pd_empirical = np.array(d_g_input) / np.array(n_g_input)

pd_empirical

array([0.00275833, 0.00592083, 0.03451667])

In [18]:
gamma_emp = norm.ppf(pd_empirical)
gamma_emp

array([-2.77520651, -2.51682787, -1.81820155])

In [20]:
-np.log(calculate_my_likelihood_arr(
            d_g_input, n_g_input, p_g, norm.pdf, np.array([0]), gamma_emp))

14.126031592781969

In [6]:
gamma_init = np.array(MM_gammas)

ML_linear_estimation = ml_estimation_linear_with_w_and_g(d_g, n_g, np.array([0.1]), gamma_init)
print("Estimated parameters with maximum likelihood method:", ML_linear_estimation[0], ML_linear_estimation[1])
print("Current function value:", ML_linear_estimation[2].fun)

Estimated parameters with maximum likelihood method: [2.99572037e-06] [-3.27228543 -2.88614722 -2.38420376 -1.72887122 -0.67703704]
Current function value: 22.117649760668396


In [ ]:
ML_linear_estimation2 = ml_estimation_linear_with_w_and_g(d_g, n_g, np.array([0.1]), gamma_init)
print("Estimated parameters with maximum likelihood method:", ML_linear_estimation[0], ML_linear_estimation[1])
print("Current function value:", ML_linear_estimation[2].fun)

In [11]:
gamma_init

array([-3.27409758, -2.88654261, -2.38422789, -1.72891114, -0.67703303])

In [12]:
w_values = np.linspace(0.0, 0.5, 6)
params_dict = {}

for w in w_values:
    ML_linear_estimation = ml_estimation_linear_with_w_and_g(d_g, n_g, np.array([w]), np.array([-3.1, -2.8, -2.3, -1.7, -0.6]), fixed_w=True)
    params_dict[w] = (ML_linear_estimation[0], ML_linear_estimation[1], ML_linear_estimation[2].fun)
    print("Estimated parameters with maximum likelihood method:", ML_linear_estimation[0], ML_linear_estimation[1])
    print("Current function value:", ML_linear_estimation[2].fun)

Estimated parameters with maximum likelihood method: [0.] [-3.27237659 -2.88615546 -2.38415936 -1.72886817 -0.67703777]
Current function value: 22.117653241620282
Estimated parameters with maximum likelihood method: [0.1] [-3.25591177 -2.87163158 -2.37220777 -1.7201529  -0.6735988 ]
Current function value: 25.570851257135974
Estimated parameters with maximum likelihood method: [0.2] [-3.16637104 -2.78811882 -2.29629282 -1.65421147 -0.62364431]
Current function value: 26.29873363554884
Estimated parameters with maximum likelihood method: [0.3] [-3.11885782 -2.75041162 -2.27159761 -1.64645764 -0.64306692]
Current function value: 26.71118980150956
Estimated parameters with maximum likelihood method: [0.4] [-3.03558021 -2.68163392 -2.22160378 -1.62097509 -0.65696757]
Current function value: 27.042978196201616


C:\Users\kissp\PycharmProjects\VasicekPDModell\src\ML_estimation.py:271: RuntimeWarning: divide by zero encountered in log
  objective_function = lambda params: -np.log(calculate_variable_changed_likelihood_arr(
C:\Users\kissp\PycharmProjects\VasicekPDModell\venv\lib\site-packages\scipy\optimize\_optimize.py:941: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):


Estimated parameters with maximum likelihood method: [0.5] [-3.1 -2.8 -2.3 -1.7 -0.6]
Current function value: inf


In [10]:
w_values = np.linspace(0.0, 0.5, 6)
params_dict = {}

for w in w_values:
    ML_linear_estimation = ml_estimation_linear_with_w_and_g(d_g, n_g, np.array([w]), gamma_init, fixed_w=True)
    params_dict[w] = (ML_linear_estimation[0], ML_linear_estimation[1], ML_linear_estimation[2].fun)
    print("Estimated parameters with maximum likelihood method:", ML_linear_estimation[0], ML_linear_estimation[1])
    print("Current function value:", ML_linear_estimation[2].fun)

Estimated parameters with maximum likelihood method: [0.] [-3.27227738 -2.88609616 -2.38419554 -1.72886231 -0.67703054]
Current function value: 22.117651533054275
Estimated parameters with maximum likelihood method: [0.1] [-3.25597653 -2.87171535 -2.37228371 -1.72023518 -0.67367521]
Current function value: 25.570850603318352
Estimated parameters with maximum likelihood method: [0.2] [-3.20618881 -2.82780797 -2.33599977 -1.69391238 -0.66333072]
Current function value: 26.27900619369894


KeyboardInterrupt: 

In [ ]:
#Make plot of the function value for different w values
function_values = [params_dict[w][2] for w in w_values]

plt.plot(w_values, function_values)
plt.xlabel("w")
plt.ylabel("Function value")
plt.title("Function value for different w values")
plt.show()

In [4]:
gamma_init = np.array(MM_gammas)

ML_linear_estimation = ml_estimation_linear_with_w_and_g(d_g, n_g, np.array([0.1]), gamma_init)
print("Estimated parameters with maximum likelihood method:", ML_linear_estimation[0], ML_linear_estimation[1])
print("Current function value:", ML_linear_estimation[2].fun)

Estimated parameters with maximum likelihood method: [ 0.19944905  0.06233111  0.02654434 -0.008197   -0.00035492] [-3.31982403 -2.91518947 -2.39704222 -1.72447598 -0.67705091]
Current function value: 24.232818680568467


In [5]:
gamma_init

array([-3.27409758, -2.88654261, -2.38422789, -1.72891114, -0.67703303])

In [ ]:
ML_linear_estimation = ml_estimation_linear_with_w_and_g(d_g, n_g, np.array([0.1]), gamma_init)
print("Estimated parameters with maximum likelihood method:", ML_linear_estimation[0], ML_linear_estimation[1])
print("Current function value:", ML_linear_estimation[2].fun)